## 준비하기
1. 아래 예제를 실행하기 위해서는 사전에 `./setup.sh` 파일을 실행하거나 `./start_notebook.sh` 로 노트북을 구동하여 의존성 라이브러리들을 미리 설치해두셔야 합니다.
2. qdrant 서버가 떠 있어야 정상 동작합니다. `./start_qdrant.sh` 를 실행하여 qdrant 서버를 띄워주세요.

## 목적:
- LangChain과 Qdrant를 사용하여 벡터 데이터 저장 및 검색
- Qdrant에 텍스트 데이터를 벡터로 변환하여 저장
- 사용자가 입력한 텍스트와 유사한 데이터를 검색

## 왜 필요한가?
- 벡터 기반 검색은 문서 검색, 챗봇, 추천 시스템 등에 필수적임
- Qdrant은 고성능 벡터 검색을 지원하는 데이터베이스임

## 주요 개념:
- **LangChain**: 자연어 처리(NLP)와 벡터 검색을 쉽게 결합할 수 있는 라이브러리
- **Qdrant**: 밀집 벡터를 저장하고 빠르게 검색할 수 있는 오픈소스 벡터 데이터베이스
- **벡터 검색**: 문서의 의미를 벡터로 변환하여 유사도를 기반으로 검색하는 방법

In [1]:
# 라이브러리 로딩 (아래 부분에서 오류가 발생하면 pip -r ./requirements.txt 로 의존성을 설치해주세요)
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

In [2]:
# Qdrant 서버 연결 (여기서 오류가 발생한다면 qdrant 서버를 띄워주세요.)
qdrant_client = QdrantClient(host="localhost", port=6333)

In [3]:
# 컬렉션 이름 설정
collection_name = "ex_1_sample_vectors"

# Qdrant 컬렉션 생성 (없을 경우 자동 생성)
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)  # 384는 임베딩 차원 수
)

/tmp/ipykernel_26400/3995727476.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [4]:
# 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
# Qdrant Vector Store 객체 생성
vectorstore = QdrantVectorStore(
    client=qdrant_client,
    collection_name=collection_name,
    embedding=embeddings
)

# 샘플 텍스트 및 메타데이터 추가
texts = [
    "Hello World",
    "LangChain is great!",
    "Vector databases are useful",
    "Qdrant is a fast vector DB"
]
metadata = [{"source": "text1"}, {"source": "text2"}, {"source": "text3"}, {"source": "text4"}]

# 벡터 저장
vectorstore.add_texts(texts=texts, metadatas=metadata)

['507063b39e004cc785bfbf49a5a62d47',
 '00fa8c1ba323403a95e970b9f816720d',
 '25547310157343d0a26c9025122eae31',
 '7e6b18ee8cd34f10a00b6a8342e892d2']

In [6]:
# 검색 실행
query_text = "LangChain"
docs = vectorstore.similarity_search(query_text, k=2)

# 검색 결과 출력
print(f"'{query_text}' 검색 결과:")
for i, doc in enumerate(docs):
    print(f"{i+1}. {doc}")

'LangChain' 검색 결과:
1. page_content='LangChain is great!' metadata={'source': 'text2', '_id': '00fa8c1b-a323-403a-95e9-70b9f816720d', '_collection_name': 'ex_1_sample_vectors'}
2. page_content='Hello World' metadata={'source': 'text1', '_id': '507063b3-9e00-4cc7-85bf-bf49a5a62d47', '_collection_name': 'ex_1_sample_vectors'}
